In [1]:
import pandas as pd
from scipy.optimize import curve_fit
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date

C:\Users\etiirine\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#read in the data
df1=pd.read_csv("RGA complete desorption measurement_no em.asc", delim_whitespace=True, index_col=False, skiprows=lambda x: x <=16)
df2=pd.read_csv("RGA complete desorption measurement_em on_grid off.asc", delim_whitespace=True, index_col=False, skiprows=lambda x: x <=16)
df3=pd.read_csv("RGA complete desorption measurement_em on_grid on.asc", delim_whitespace=True, index_col=False, skiprows=lambda x: x <=16)
df1.head()

,ScanData,1
0,0.00,3.586220e-13
1,0.03,3.436770e-13
2,0.06,4.251090e-13
3,0.09,3.409520e-13
4,0.13,2.554780e-13


In [3]:
df2.head()

df1 = df1.rename(columns={'ScanData': 'Mass fraction', '1': 'Ion current'})
df2 = df2.rename(columns={'ScanData': 'Mass fraction', '1': 'Ion current'})
df3 = df3.rename(columns={'ScanData': 'Mass fraction', '1': 'Ion current'})

df3.head()

,Mass fraction,Ion current
0,0.00,4.589030e-13
1,0.03,4.086860e-13
2,0.06,5.733860e-13
3,0.09,4.328080e-13
4,0.13,5.049220e-13


In [4]:
#Plotting the all the spectrums overlaid on top
plt.figure(figsize=(14,8))
plt.plot((df1["Mass fraction"]),df1['Ion current'],marker=".", markersize=4,label='Emission off')
plt.plot((df2["Mass fraction"]),df2['Ion current'],marker=".", markersize=4,label='Emission on, grid transparent')
plt.plot((df3["Mass fraction"]),df3['Ion current'],marker=".", markersize=4,label='Emission on, grid repelling')
plt.xlabel('Mass fraction (u)')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("RGA scan - full spectrum")
plt.xticks(np.arange(0,50,1.0))
plt.yscale('log')
plt.ylim(7e-13,1e-9)

(7e-13, 1e-09)

In [5]:
#Effect of turning the emission on
emission_sig=df2["Ion current"]-df1["Ion current"]
df2["Emission sig"]=emission_sig


In [6]:
#Plotting Em on, grid transp. - em off
plt.figure(figsize=(14,8))
plt.plot((df1["Mass fraction"]),emission_sig,marker=".", markersize=4,label='Subtracted signal')
plt.xlabel('Mass fraction (u)')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("Emission on, grid transp - Emission off")
plt.xticks(np.arange(0,50,1.0))
plt.yscale('log')
plt.ylim(7e-13,1e-9)

(7e-13, 1e-09)

In [7]:
#Effect of the EGA
grid_sig=df3["Ion current"]-df2["Ion current"]

In [8]:
#Plotting  Emission on, grid repelling - Emission on, grid transp.
plt.figure(figsize=(12,6))
plt.plot((df2["Mass fraction"]),grid_sig,marker=".", markersize=4,label='Subtracted signal')
plt.xlabel('Mass fraction (u)')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("Emission on, grid repelling - Emission on, grid transparent")
plt.xticks(np.arange(0,50,1.0))
plt.yscale('log')
plt.ylim(7e-13,1e-10)

(7e-13, 1e-10)

In [9]:
#sample ESD= Emission on, grid transp - emission on,grid repelling
df2["Sample sig"]=df2["Ion current"]-df3["Ion current"]
df2[df2 < 0] = 0
df2=df2.iloc[::-1]
df2.head()

,Mass fraction,Ion current,Emission sig,Sample sig
1599,49.97,6.687520e-13,0.000000e+00,5.126000e-14
1598,49.94,8.170670e-13,5.052000e-13,7.137490e-13
1597,49.91,9.966400e-13,6.771540e-13,6.117550e-13
1596,49.88,6.715880e-13,4.637130e-13,0.000000e+00
1595,49.84,7.055180e-13,1.430980e-13,1.878570e-13


In [38]:
#Plotting Emission on, grid transp. - Emission on, grid repelling
#essentially total ESD= EGA ESD + sample ESD
#sample ESD= Emission on, grid transp - emission on,grid repelling
plt.figure(figsize=(12,6))
plt.plot((df2["Mass fraction"]),df2["Sample sig"],marker=".", markersize=4,label='Subtracted signal')
plt.xlabel('Mass fraction (u)')
plt.ylabel('Ion current (A)')
plt.legend(loc="upper right")
plt.title("Differential mass spectrum of ESD from the copper sample")
plt.xticks(np.arange(0,50,2.0),rotation='vertical')
plt.yscale('log')
plt.ylim(7e-13,1e-10)

(7e-13, 1e-10)

In [11]:
#Desorption yield
#First, reading in the Emission current
I_em=pd.read_csv("complete desorption measurement", sep="\t")
I_em.drop(columns=['Full_Range_Turbo','Pirani_1','Injection','XHVTrap'], inplace=True)
I_em.drop(labels=[0,1], axis=0)


,Time,Penning_1,Penning_2,I_Em,Live comments
2,12/10/2022 14:04:23,2.600000e-10,3.600000e-10,1.768900e-12,NaN
3,12/10/2022 14:04:28,2.600000e-10,3.600000e-10,2.357900e-12,NaN
4,12/10/2022 14:04:34,2.600000e-10,3.600000e-10,3.879300e-12,NaN
5,12/10/2022 14:04:40,2.600000e-10,3.600000e-10,4.801500e-12,NaN
6,12/10/2022 14:04:46,2.600000e-10,3.600000e-10,9.268000e-13,NaN
...,...,...,...,...,...
1668,12/10/2022 16:28:12,1.400000e-09,3.200000e-09,1.097650e-10,NaN
1669,12/10/2022 16:28:18,1.400000e-09,3.200000e-09,1.151130e-10,NaN
1670,12/10/2022 16:28:24,1.400000e-09,3.200000e-09,1.112000e-10,NaN
1671,12/10/2022 16:28:29,1.400000e-09,3.200000e-09,1.156240e-10,NaN


In [12]:
#adding an elapsed time column
timestamp=[]
for i in I_em["Time"]:
    timestamp.append(i)
#making the column into a datetime object
timestamps=pd.to_datetime(I_em["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
I_em.insert(1,"Elapsed time",runtime)
I_em.drop(index=I_em.index[0], 
        axis=0, 
        inplace=True)

I_em.head()

,Time,Elapsed time,Penning_1,Penning_2,I_Em,Live comments
1,12/10/2022 14:04:17,5.0,2.600000e-10,3.600000e-10,8.016000e-13,NaN
2,12/10/2022 14:04:23,11.0,2.600000e-10,3.600000e-10,1.768900e-12,NaN
3,12/10/2022 14:04:28,16.0,2.600000e-10,3.600000e-10,2.357900e-12,NaN
4,12/10/2022 14:04:34,22.0,2.600000e-10,3.600000e-10,3.879300e-12,NaN
5,12/10/2022 14:04:40,28.0,2.600000e-10,3.600000e-10,4.801500e-12,NaN


In [13]:
#Plotting the I_em
plt.figure(figsize=(10,6))
plt.plot(I_em["Elapsed time"]/3600,I_em["I_Em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (h)')
plt.ylabel('Emission current (A)')
plt.legend()
plt.title("Emission current over time")
plt.yscale('log')


In [14]:
#Plotting the pressure of PE2
plt.figure(figsize=(10,6))
plt.plot(I_em["Elapsed time"]/3600,I_em["Penning_2"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (h)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Chamber pressure over time")
plt.yscale('log')
plt.ylim(1e-10,1e-8)

(1e-10, 1e-08)

In [15]:
#Reading in the HiVolta measurement file to get the grid current
hv=pd.read_csv("HiVolta req complete desorption measurement", sep=",", index_col=False)
hv.columns=["Date","Time","Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon1","Imon2","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"] 
hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
print(hv["Imon1"])
hv_grid=[element * 1e-6 for element in hv["Imon1"]]
print(hv_grid)

0       0.029
1       0.029
2       0.029
3       0.029
4       0.029
        ...  
7262    0.296
7263    0.296
7264    0.296
7265    0.296
7266    0.296
Name: Imon1, Length: 7267, dtype: float64
[2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.9e-08, 2.

In [36]:
#Gas species detected desorbing from the Cu sample include H2 and CO2, others in marginal amounts
#For this calculation it is assumed that H2 and CO2 make up 100% of the total I_em
#step 1: pressure difference
delta_p2=(I_em["Penning_2"].max()-I_em["Penning_2"].min())*100
delta_p1=(I_em["Penning_1"].max()-I_em["Penning_1"].min())*100

print("Pressure delta (PE2) in Pa:",delta_p)
#step 2 constants
cond_h2=37.91/1000 #in m3/s
cond_co2=8.08/1000
cond_h2o=12.64/1000
S_eff=42/1000
k_B=1.38e-23
T=296
q_e=1.6e-19

Pressure delta (PE2) in Pa: 5.24e-07


In [17]:
#rather than using peak values, one should integrate
#first find the peak indexes 
h2_peak=df2.index[(df2["Mass fraction"]>1.5) & (df2["Mass fraction"]<2.5)].tolist()
print(h2_peak)
co2_peak=df2.index[(df2["Mass fraction"]>33.5) & (df2["Mass fraction"]<44.5)].tolist()
co2_peak=co2_peak + df2.index[(df2["Mass fraction"]>21.5) & (df2["Mass fraction"]<22.5)].tolist()

print(co2_peak)
h2o_peak=df2.index[(df2["Mass fraction"]>17.5) & (df2["Mass fraction"]<19.5)].tolist()

[79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49]
[1423, 1422, 1421, 1420, 1419, 1418, 1417, 1416, 1415, 1414, 1413, 1412, 1411, 1410, 1409, 1408, 1407, 1406, 1405, 1404, 1403, 1402, 1401, 1400, 1399, 1398, 1397, 1396, 1395, 1394, 1393, 1392, 1391, 1390, 1389, 1388, 1387, 1386, 1385, 1384, 1383, 1382, 1381, 1380, 1379, 1378, 1377, 1376, 1375, 1374, 1373, 1372, 1371, 1370, 1369, 1368, 1367, 1366, 1365, 1364, 1363, 1362, 1361, 1360, 1359, 1358, 1357, 1356, 1355, 1354, 1353, 1352, 1351, 1350, 1349, 1348, 1347, 1346, 1345, 1344, 1343, 1342, 1341, 1340, 1339, 1338, 1337, 1336, 1335, 1334, 1333, 1332, 1331, 1330, 1329, 1328, 1327, 1326, 1325, 1324, 1323, 1322, 1321, 1320, 1319, 1318, 1317, 1316, 1315, 1314, 1313, 1312, 1311, 1310, 1309, 1308, 1307, 1306, 1305, 1304, 1303, 1302, 1301, 1300, 1299, 1298, 1297, 1296, 1295, 1294, 1293, 1292, 1291, 1290, 1289, 1288, 1287, 1286, 1285, 1284, 1283, 1282, 1281, 1280, 1279, 1278

In [18]:
#integration
h2_integral=np.trapz(df2["Emission sig"][h2_peak])
co2_integral=np.trapz(df2["Emission sig"][co2_peak])
h2o_integral=np.trapz(df2["Emission sig"][h2o_peak])

print(h2_integral, co2_integral, h2o_integral)
#co2_integral=
#h2o_integral=

5.280983170000001e-09 1.04181652746e-09 4.2001036609999993e-10


In [19]:
#Probably a mistake here

#fractions
#assuming H2, H2O and CO2 (and its meaningful fragments C, O) make up 100% of the desorbing gas
#x_sum=df2["Ion current"].max()+df2["Ion current"].nlargest(5).values[0]+df2["Ion current"].nlargest(5).values[1]+df2["Ion current"].nlargest(5).values[2]+df2["Ion current"].nlargest(5).values[3]+df2["Ion current"].nlargest(5).values[4]
#x_sum=df2["Ion current"].sum(axis=0)
#print(x_sum)
#x_co2=df2["Ion current"].max()/x_sum
#x_h2=df2["Ion current"].nlargest(2).values[-1]/x_sum
#print(df2["Ion current"].nlargest(2).values[-1])
#x_h2o=df2["Ion current"].nlargest(5).values[4]/x_sum
#print("Fractions:",x_co2,"\n",x_h2,"\n", x_h2o)
#Multiplying maximum EGA emission current (grid transparent) - grid current by the fractions
#w_co2=x_co2*(I_em["I_Em"].max())
#w_h2=x_h2*(I_em["I_Em"].max())
#w_h2o=x_h2o*(I_em["I_Em"].max())
#print("weights:",w_co2,"\n",w_h2,"\n", w_h2o)

In [33]:
#fractions
#assuming H2, H2O and CO2 (and its meaningful fragments C, O) make up 100% of the desorbing gas
x_sum=h2_integral+ co2_integral + h2o_integral
#x_sum=df2["Ion current"].sum(axis=0)
print(x_sum)
x_co2=co2_integral/x_sum
x_h2=h2_integral/x_sum
x_h2o=h2o_integral/x_sum
print("Fractions:",x_co2,"\n",x_h2,"\n", x_h2o)
#Multiplying maximum EGA emission current (grid transparent) - grid current by the fractions
w_co2=x_co2*(I_em["I_Em"].max())
w_h2=x_h2*(I_em["I_Em"].max())
w_h2o=x_h2o*(I_em["I_Em"].max())
print("weighted emission currents:",w_co2,"\n",w_h2,"\n", w_h2o)

6.742810063560001e-09
Fractions: 0.1545077671830418 
 0.7832021249626896 
 0.06229010785426856
weighted emission currents: 1.5758865206067165e-07 
 7.988191753344456e-07 
 6.353217260488268e-08


In [37]:
#calculating the desorption yields
eta_co2=((delta_p2-delta_p1)*cond_co2*q_e)/(k_B*T*w_co2)
eta_h2=((delta_p2-delta_p1)*cond_h2*q_e)/(k_B*T*w_h2)
eta_h2o=((delta_p2-delta_p1)*cond_h2o*q_e)/(k_B*T*w_h2o)
print("desorption yields:","\n","co2:",eta_co2,"\n","h2:",eta_h2,"\n","h2o", eta_h2o)

desorption yields: 
 co2: 0.5824164100615069 
 h2: 0.5390790894446157 
 h2o 2.2599589460590352


In [22]:
#finding the fluxes for graphing
I_sum=df2["Ion current"].sum(axis=0)
ar=[df2["Ion current"].nlargest(5).values[0],df2["Ion current"].nlargest(5).values[1],df2["Ion current"].nlargest(5).values[4]]
dp=[]
for i in ar:
    dp.append(i/I_sum)

fluxes=[element * S_eff for element in dp]
print(fluxes)

[0.0012275213730116473, 0.0012028953777781328, 0.0011461220600720776]


In [23]:
#Plotting
#desorption yield
plt.figure(figsize=(10,6))
plt.plot(fluxes[0],eta_co2,marker=".", markersize=20, label="CO2")
plt.plot(fluxes[1],eta_h2,marker=".", markersize=20, label="H2")
plt.plot(fluxes[2],eta_h2o,marker=".", markersize=20, label="H2O")


plt.xlabel('Flux due to ESD (Pa*m3/s)')
plt.ylabel('Desorption yield (mol/el)')
plt.legend()
plt.title("Desorption yield vs flux")
plt.yscale('log')

# Primary desorption calculation method
### Elena's formula for the cryo setup
$$
\eta=\frac{C_{H2}\cdot (\Delta p_1 - \Delta p_2)q_e}{k_B\cdot T \cdot I_e}
$$
In the case of the RT setup, we dont know the exact conductance due to not having a calibrated orifice. And the $\Delta p$ in this case is the difference of partial pressures (for the gas in question) e.g grid transparent - grid repelling. Because we use the partial pressures, $I_e$ here is the total emission current.
### RT setup:
So the formula in the case of the RT setup would be:
$$
\eta_{gas}=\frac{Q\cdot q_e}{k_B \cdot T \cdot I_{e, tot}}=\frac{\Delta p_{partial} \cdot S_{gas}\cdot q_e}{k_B \cdot T \cdot I_{e, tot}}
$$
where $\Delta p_{partial}$ would be:
$$
\Delta p_{partial}=p_{par, transm.}-p_{par, repel.}
$$

and
$$
p_{par}=\frac{ref_{peak}\cdot p_{tot}}{\Sigma_{refpeaks}}
$$

In [32]:
#calculating partial pressures
#assuming here that h2, h2o and co2 make up 100% of the desorbed gas
#refpeaks: h2- 2 AMU, h2o - 18 AMU, CO2 - 44 AMU
h2_transm=df2["Ion current"][h2_peak]
h2_repel=df3["Ion current"]

0.42220788837597484
